## Training for language model

This contains the code that I used to train llms (I am using the Llama 3.1 8B Base model) for recipie generation. The code in this notebook is adapted from the google collab notebook used in [this](https://www.youtube.com/watch?v=Us5ZFp16PaU) tutorial.

In [ ]:
# Mount google drive

from google.colab import drive
drive.mount('/content/gdrive')
nbdir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
# Login to hugging face

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Install modules

!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Instal

In [ ]:
# Import modules

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
# Set up the model and load in 8 bit

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

## Set up the model parameters and lora

In [ ]:
# Freeze the original model weights

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
# Set up the lora adapters

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6815744 || all params: 8037076992 || trainable%: 0.08480376642881861


## Preprocess the data for training

I am using the "recipie_short" dataset from hugging face. This dataset contains 350 records of recipies. The first step of preprocessing is to select a subset of the dataset for training. The next step is to discard uncessary columns from the dataset.

In [ ]:
# Data preprocessing

import transformers
from datasets import load_dataset
data = load_dataset("rk404/recipe_short")

Generating train split:   0%|          | 0/350000 [00:00<?, ? examples/s]

In [ ]:
# Use a portion of the data for training

data = data['train'].select(range(len(data['train']) // 20))

data


Dataset({
    features: ['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source', 'NER'],
    num_rows: 17500
})

In [ ]:
# Discard uncessary columns from the dataset

data = data.select_columns(["title", "ingredients", "directions", "NER"])

data

Dataset({
    features: ['title', 'ingredients', 'directions', 'NER'],
    num_rows: 17500
})

I then merge the columns and map them to the prediction.

In [ ]:
def merge_columns(example):
    example["prediction"] = example["NER"] + " ->: " + "title: " + str(example["title"]) + "\n" + "ingredients: " + str(example["ingredients"]) + "\n" + "directions: " + str(example["directions"])
    return example

data = data.map(merge_columns)
data["prediction"][0]

Map:   0%|          | 0/17500 [00:00<?, ? examples/s]

'["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"] ->: title: No-Bake Nut Cookies\ningredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\ndirections: ["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]'

In [ ]:
data[0]

{'title': 'No-Bake Nut Cookies',
 'ingredients': '["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]',
 'directions': '["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]',
 'NER': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]',
 'prediction': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"] ->: title: No-Bake Nut Cookies\ningredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or mar

In [ ]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/17500 [00:00<?, ? examples/s]

## Train the model

In [ ]:
# Set the pad token

tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Train the model

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=50,
        max_steps=500,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=20,
        output_dir="/content/gdrive/MyDrive/recipie-lora-llama-2"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
20,1.577600
40,1.212400
60,1.099700
80,1.096100
100,1.049900
120,1.038400
140,0.988100
160,1.023600
180,1.000400
200,0.990100


TrainOutput(global_step=500, training_loss=1.0259752540588378, metrics={'train_runtime': 3182.3773, 'train_samples_per_second': 0.628, 'train_steps_per_second': 0.157, 'total_flos': 1.881745058738995e+16, 'train_loss': 1.0259752540588378, 'epoch': 0.11428571428571428})

In [ ]:
model.push_to_hub("yr82/recipie-lora-llama-2",
                  use_auth_token=True,
                  commit_message="lora training - llama 3.1",
                  private=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:892: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yr82/recipie-lora-llama-2/commit/00065e781c5ddcd1bfbe8b7734e56a33b8896f32', commit_message='lora training - llama 3.1', commit_description='', oid='00065e781c5ddcd1bfbe8b7734e56a33b8896f32', pr_url=None, pr_revision=None, pr_num=None)

## Infernce

In [ ]:
# Load the adapter

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "yr82/recipie-lora-llama-2"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [ ]:
# Test with something from the training set

batch = tokenizer("[\"brown sugar\", \"milk\", \"vanilla\", \"nuts\", \"butter\", \"bite size shredded rice biscuits\"]" + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch,  max_new_tokens=512, min_new_tokens=64, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=False))

<ipython-input-21-34fcad2483d4>:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1902: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 <|begin_of_text|>["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"] ->:  title: Rice Krispie Treats
ingredients: ["1 c. brown sugar", "1 c. milk", "1 tsp. vanilla", "1 c. nuts", "1/2 c. butter", "bite size shredded rice biscuits"]
directions: ["Mix sugar, milk and butter in saucepan.", "Boil 1 minute.", "Add vanilla and nuts.", "Pour over rice Krispies.", "Mix well.", "Pour into buttered pan.", "Cool."]<|end_of_text|>


In [ ]:
# Test with something random

batch = tokenizer("[\"chocolate\", \"flour\", \"eggs\"]" + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch,  max_new_tokens=512, min_new_tokens=64, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=False))

<ipython-input-19-a2f71297b5ca>:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1902: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 <|begin_of_text|>["chocolate", "flour", "eggs"] ->:  title: Chocolate Cake
ingredients: ["1 c. Hershey's cocoa", "1 c. flour", "3 eggs"]
directions: ["Mix all ingredients together.", "Pour into a greased 9 x 13-inch pan.", "Bake at 350\u00b0 for 30 minutes."]<|end_of_text|>


In [ ]:
def format_list(items):
    formatted_list = ', '.join([f'"{item}"' for item in items])
    return f'[{formatted_list}]'

In [ ]:
food = ["tomatoes",
 "tomatoes",
 "oatmeal",
 "tomatoes",
 "mozzarella",
 "eggs",
 "beans",
 "garbanzo",
 "sprouted",
 "bag_jazz",
 "bananas",
 "creany",
 "peanut_butter",
 "pita_bread"
]

food_list = format_list(food)

In [ ]:
print(food_list)

["tomatoes", "tomatoes", "oatmeal", "tomatoes", "mozzarella", "eggs", "beans", "garbanzo", "sprouted", "bag_jazz", "bananas", "creany", "peanut_butter", "pita_bread"]


In [ ]:
# Test some data from receipts

batch = tokenizer(food_list + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

<ipython-input-43-d656b3f1d450>:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1902: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 ["tomatoes", "tomatoes", "oatmeal", "tomatoes", "mozzarella", "eggs", "beans", "garbanzo", "sprouted", "bag_jazz", "bananas", "creany", "peanut_butter", "pita_bread"] ->:  title: The Best Breakfast
ingredients: ["tomatoes", "tomatoes", "oatmeal", "tomatoes", "mozzarella", "eggs", "beans", "garbanzo", "sprouted", "bag_jazz",


In [ ]:
# Remove duplicates and use only the first 5 items from the list

unique_food = list(dict.fromkeys(food))

unique_food = unique_food[:5]

food_list = format_list(unique_food)

In [ ]:
print(unique_food)

['tomatoes', 'oatmeal', 'mozzarella', 'eggs', 'beans']


In [ ]:
batch = tokenizer(food_list + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=False, eos_token_id=tokenizer.eos_token_id))

<ipython-input-18-3df50dc6c9e2>:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




 <|begin_of_text|>["tomatoes", "oatmeal", "mozzarella", "eggs", "beans"] ->:  title: Baked Tomatoes
ingredients: ["2 c. tomatoes", "1/2 c. oatmeal", "1/2 c. mozzarella cheese", "2 eggs", "1 can beans"]
directions: ["Cut tomatoes in


In [ ]:
# Generate recipie

batch = tokenizer(food_list + " ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=512, min_new_tokens=64, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

<ipython-input-16-38d3787dac53>:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1902: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
sections = output_text.split("\n")

for section in sections:
  print(section)

["tomatoes", "oatmeal", "mozzarella", "eggs", "beans"] ->:  title: Baked Tomatoes
ingredients: ["2 c. tomatoes", "1/2 c. oatmeal", "1/2 c. mozzarella cheese", "2 eggs", "1 can beans"]
directions: ["Cut tomatoes in half.", "Mix oatmeal, cheese and eggs.", "Fill tomatoes with mixture.", "Bake at 350\u00b0 for 30 minutes.", "Top with beans and bake 10 minutes more."]
